### Extraction Phase

In [5]:
# Installing Necessary Libraries

%pip install pyarrow
%pip install azure-storage-blob
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Importing Necessary Packages
import pandas as pd
import os
import io
from azure.storage.blob import BlobServiceClient, BlobClient
from dotenv import load_dotenv

In [7]:
# Reading the Data from the source
nycpayroll2020_df = pd.read_csv(r'Data\nycpayroll_2020.csv')
nycpayroll2021_df = pd.read_csv(r'Data\nycpayroll_2021.csv')
agency_df = pd.read_csv(r'Data\AgencyMaster.csv')
employee_df = pd.read_csv(r'Data\EmpMaster.csv')
title_df = pd.read_csv(r'Data\TitleMaster.csv')

In [8]:
nycpayroll2020_df.head()

,FiscalYear,PayrollNumber,AgencyID,AgencyName,EmployeeID,LastName,FirstName,AgencyStartDate,WorkLocationBorough,TitleCode,TitleDescription,LeaveStatusasofJune30,BaseSalary,PayBasis,RegularHours,RegularGrossPaid,OTHours,TotalOTPaid,TotalOtherPay
0,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,10001,GEAGER,VERONICA,9/12/2016,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
1,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,149612,ROTTA,JONATHAN,9/16/2013,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
2,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,206583,WILSON II,ROBERT,4/30/2018,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
3,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,199874,WASHINGTON,MORIAH,3/18/2019,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,87900.95,0.0,0.0,-3202.74
4,2020,17,2120,OFFICE OF EMERGENCY MANAGEMENT,58036,KRAWCZYK,AMANDA,5/15/2017,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,83976.54,0.0,0.0,0.00


In [9]:
nycpayroll2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FiscalYear             100 non-null    int64  
 1   PayrollNumber          100 non-null    int64  
 2   AgencyID               100 non-null    int64  
 3   AgencyName             100 non-null    object 
 4   EmployeeID             100 non-null    int64  
 5   LastName               100 non-null    object 
 6   FirstName              100 non-null    object 
 7   AgencyStartDate        100 non-null    object 
 8   WorkLocationBorough    100 non-null    object 
 9   TitleCode              100 non-null    int64  
 10  TitleDescription       100 non-null    object 
 11  LeaveStatusasofJune30  100 non-null    object 
 12  BaseSalary             100 non-null    float64
 13  PayBasis               100 non-null    object 
 14  RegularHours           100 non-null    float64
 15  Regular

In [10]:
nycpayroll2020_df.shape

(100, 19)

### Data Cleaning and Transfromation

In [11]:
 # Converting the FiscalYear to datetime
nycpayroll2020_df['FiscalYear'] = pd.to_datetime(nycpayroll2020_df['FiscalYear'], format='%Y') 

# Converting the AgencyStartDate to datetime
nycpayroll2020_df['AgencyStartDate'] = pd.to_datetime(nycpayroll2021_df['AgencyStartDate'], format='%m/%d/%Y')

In [12]:
nycpayroll2020_df.head()

,FiscalYear,PayrollNumber,AgencyID,AgencyName,EmployeeID,LastName,FirstName,AgencyStartDate,WorkLocationBorough,TitleCode,TitleDescription,LeaveStatusasofJune30,BaseSalary,PayBasis,RegularHours,RegularGrossPaid,OTHours,TotalOTPaid,TotalOtherPay
0,2020-01-01,17,2120,OFFICE OF EMERGENCY MANAGEMENT,10001,GEAGER,VERONICA,2018-02-26,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
1,2020-01-01,17,2120,OFFICE OF EMERGENCY MANAGEMENT,149612,ROTTA,JONATHAN,2019-08-12,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
2,2020-01-01,17,2120,OFFICE OF EMERGENCY MANAGEMENT,206583,WILSON II,ROBERT,2018-02-26,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
3,2020-01-01,17,2120,OFFICE OF EMERGENCY MANAGEMENT,199874,WASHINGTON,MORIAH,2010-08-02,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,87900.95,0.0,0.0,-3202.74
4,2020-01-01,17,2120,OFFICE OF EMERGENCY MANAGEMENT,58036,KRAWCZYK,AMANDA,2007-01-16,BROOKLYN,40447,EMERGENCY PREPAREDNESS MANAGER,ACTIVE,86005.0,per Annum,1820.0,83976.54,0.0,0.0,0.00


In [13]:

nycpayroll2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   FiscalYear             100 non-null    datetime64[ns]
 1   PayrollNumber          100 non-null    int64         
 2   AgencyID               100 non-null    int64         
 3   AgencyName             100 non-null    object        
 4   EmployeeID             100 non-null    int64         
 5   LastName               100 non-null    object        
 6   FirstName              100 non-null    object        
 7   AgencyStartDate        100 non-null    datetime64[ns]
 8   WorkLocationBorough    100 non-null    object        
 9   TitleCode              100 non-null    int64         
 10  TitleDescription       100 non-null    object        
 11  LeaveStatusasofJune30  100 non-null    object        
 12  BaseSalary             100 non-null    float64       
 13  PayBas

In [14]:
nycpayroll2020_df.columns

Index(['FiscalYear', 'PayrollNumber', 'AgencyID', 'AgencyName', 'EmployeeID',
       'LastName', 'FirstName', 'AgencyStartDate', 'WorkLocationBorough',
       'TitleCode', 'TitleDescription', 'LeaveStatusasofJune30', 'BaseSalary',
       'PayBasis', 'RegularHours', 'RegularGrossPaid', 'OTHours',
       'TotalOTPaid', 'TotalOtherPay'],
      dtype='object')

In [15]:
# Creating Employee Table
employee = nycpayroll2020_df[['EmployeeID','LastName', 'FirstName', 'TitleDescription','AgencyName']].copy().drop_duplicates().reset_index(drop=True)


In [16]:
employee.head()

,EmployeeID,LastName,FirstName,TitleDescription,AgencyName
0,10001,GEAGER,VERONICA,EMERGENCY PREPAREDNESS MANAGER,OFFICE OF EMERGENCY MANAGEMENT
1,149612,ROTTA,JONATHAN,EMERGENCY PREPAREDNESS MANAGER,OFFICE OF EMERGENCY MANAGEMENT
2,206583,WILSON II,ROBERT,EMERGENCY PREPAREDNESS MANAGER,OFFICE OF EMERGENCY MANAGEMENT
3,199874,WASHINGTON,MORIAH,EMERGENCY PREPAREDNESS MANAGER,OFFICE OF EMERGENCY MANAGEMENT
4,58036,KRAWCZYK,AMANDA,EMERGENCY PREPAREDNESS MANAGER,OFFICE OF EMERGENCY MANAGEMENT


In [17]:
employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   EmployeeID        100 non-null    int64 
 1   LastName          100 non-null    object
 2   FirstName         100 non-null    object
 3   TitleDescription  100 non-null    object
 4   AgencyName        100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [18]:
employee.shape

(100, 5)

In [19]:
nycpayroll2020_df.columns

Index(['FiscalYear', 'PayrollNumber', 'AgencyID', 'AgencyName', 'EmployeeID',
       'LastName', 'FirstName', 'AgencyStartDate', 'WorkLocationBorough',
       'TitleCode', 'TitleDescription', 'LeaveStatusasofJune30', 'BaseSalary',
       'PayBasis', 'RegularHours', 'RegularGrossPaid', 'OTHours',
       'TotalOTPaid', 'TotalOtherPay'],
      dtype='object')

In [20]:
# Creating Agency Table
agency = nycpayroll2020_df[['AgencyID','AgencyName','AgencyStartDate','WorkLocationBorough']].copy().drop_duplicates().reset_index(drop=True)

In [21]:
agency

,AgencyID,AgencyName,AgencyStartDate,WorkLocationBorough
0,2120,OFFICE OF EMERGENCY MANAGEMENT,2018-02-26,BROOKLYN
1,2120,OFFICE OF EMERGENCY MANAGEMENT,2019-08-12,BROOKLYN
2,2120,OFFICE OF EMERGENCY MANAGEMENT,2010-08-02,BROOKLYN
3,2120,OFFICE OF EMERGENCY MANAGEMENT,2007-01-16,BROOKLYN
4,2120,OFFICE OF EMERGENCY MANAGEMENT,2011-04-17,BROOKLYN
...,...,...,...,...
92,2122,OFFICE OF MANAGEMENT & BUDGET,2004-03-04,MANHATTAN
93,2122,OFFICE OF MANAGEMENT & BUDGET,2016-05-31,MANHATTAN
94,2122,OFFICE OF MANAGEMENT & BUDGET,1990-08-19,MANHATTAN
95,2122,OFFICE OF MANAGEMENT & BUDGET,1998-06-30,MANHATTAN


In [22]:
agency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   AgencyID             97 non-null     int64         
 1   AgencyName           97 non-null     object        
 2   AgencyStartDate      97 non-null     datetime64[ns]
 3   WorkLocationBorough  97 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.2+ KB


In [23]:
agency.shape

(97, 4)

In [24]:
# Creating Title Table
title = nycpayroll2020_df[['TitleCode', 'TitleDescription']].copy().drop_duplicates().reset_index(drop=True)

In [25]:
title.head()

,TitleCode,TitleDescription
0,40447,EMERGENCY PREPAREDNESS MANAGER
1,40285,COMMISSIONER OF EMERGENCY MANAGEMENT
2,40448,EMERGENCY PREPAREDNESS SPECIALIST
3,40291,COMMUNITY ASSOCIATE
4,40362,DEPUTY COMMISSIONER


In [26]:
title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TitleCode         12 non-null     int64 
 1   TitleDescription  12 non-null     object
dtypes: int64(1), object(1)
memory usage: 324.0+ bytes


In [27]:
# Merging the DataFrames
payroll_fact = nycpayroll2020_df.merge(employee, on=['EmployeeID','LastName', 'FirstName', 'TitleDescription','AgencyName'], how='left') \
                                .merge(agency, on=['AgencyID','AgencyName','AgencyStartDate','WorkLocationBorough'], how='left') \
                                .merge(title, on=['TitleCode', 'TitleDescription'], how='left') \
                [['FiscalYear','PayrollNumber','AgencyID','EmployeeID','TitleCode','LeaveStatusasofJune30','BaseSalary','PayBasis','RegularHours','RegularGrossPaid','OTHours', 'TotalOTPaid', 'TotalOtherPay']] 

payroll_fact.index.name = 'PayrollFactID'
payroll_fact = payroll_fact.reset_index()

payroll_fact.head()


,PayrollFactID,FiscalYear,PayrollNumber,AgencyID,EmployeeID,TitleCode,LeaveStatusasofJune30,BaseSalary,PayBasis,RegularHours,RegularGrossPaid,OTHours,TotalOTPaid,TotalOtherPay
0,0,2020-01-01,17,2120,10001,40447,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
1,1,2020-01-01,17,2120,149612,40447,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
2,2,2020-01-01,17,2120,206583,40447,ACTIVE,86005.0,per Annum,1820.0,84698.21,0.0,0.0,0.00
3,3,2020-01-01,17,2120,199874,40447,ACTIVE,86005.0,per Annum,1820.0,87900.95,0.0,0.0,-3202.74
4,4,2020-01-01,17,2120,58036,40447,ACTIVE,86005.0,per Annum,1820.0,83976.54,0.0,0.0,0.00


In [28]:
payroll_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   PayrollFactID          100 non-null    int64         
 1   FiscalYear             100 non-null    datetime64[ns]
 2   PayrollNumber          100 non-null    int64         
 3   AgencyID               100 non-null    int64         
 4   EmployeeID             100 non-null    int64         
 5   TitleCode              100 non-null    int64         
 6   LeaveStatusasofJune30  100 non-null    object        
 7   BaseSalary             100 non-null    float64       
 8   PayBasis               100 non-null    object        
 9   RegularHours           100 non-null    float64       
 10  RegularGrossPaid       100 non-null    float64       
 11  OTHours                100 non-null    float64       
 12  TotalOTPaid            100 non-null    float64       
 13  TotalO

In [29]:
payroll_fact.shape

(100, 14)

In [ ]:
# saving the data to csv
employee.to_csv(r'processed_data\employee.csv', index=False)
agency.to_csv(r'processed_data\agency.csv', index=False)
title.to_csv(r'processed_data\title.csv', index=False)
payroll_fact.to_csv(r'processed_data\payroll_fact.csv', index=False)

### Data Loading

In [ ]:
# Setting up Azure Blob Storage connection

load_dotenv()

connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

container_name = os.getenv('AZURE_STORAGE_CONTAINER_NAME')
container_client = blob_service_client.get_container_client(container_name)



In [ ]:
# Create a function to upload files to the Azure Blob Storage
def upload_to_azure_blob(df, container_client, blob_name):
    buffer = io.BytesIO()
    df.to_parquet(buffer, index=False)
    buffer.seek(0)
    blob_client = container_client.get_blob_client(blob_name)
    blob_client.upload_blob(buffer, blob_type="BlockBlob", overwrite=True)
    print(f'{blob_name} uploaded to Azure Blob Storage')
    

In [ ]:
upload_to_azure_blob(employee, container_client, 'processed_data/employee.parquet')
upload_to_azure_blob(agency, container_client, 'processed_data/agency.parquet')
upload_to_azure_blob(title, container_client, 'processed_data/title.parquet')
upload_to_azure_blob(payroll_fact, container_client, 'processed_data/payroll_fact.parquet')